# **Maestría en Inteligencia Artificial Aplicada**
## **Curso: Inteligencia Artificial y Aprendizaje Automático**
### Tecnológico de Monterrey
### Prof. Luis Eduardo Falcón Morales

## **Actividad de la Semana 6**
### **Árboles de decisión y bosque aleatorio.**

**Nombres y matrículas de los integrantes del equipo:**

* Jonathan Garza Bennet | 
* Diego Fernando Guerra Burgos | 
* Fernando Maytorena Espinosa de los Monteros | A01110946 
* José Francisco Miranda García |  

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [1]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# **Ejercicio-1.** 

In [ ]:
None

# **Ejercicio-2.**

In [ ]:
None

# **Ejercicio-3.**

In [ ]:
None

# **Ejercicio-4.**

In [ ]:
None

# **Ejercicio-5.**

In [ ]:
None

# **Ejercicio-6.**

In [ ]:
None

### **Fin de la Actividad de la semana 6.**

---